# CoViD-19 Swedish QA
I decided to scratch a small itch I've had for a while now - creating a search engine using an unsupervised approach. The final product, or the first iteration rather, ended up pretty good and I wanted to share what I've done so far.

### Introduction to the problem and requirements
An unsupervised approach where we never edit the data nor supply any manually annotated data? Every Data Scientist dream I suppose. There's a reason as of why supervised approaches generally result in better performance but there is some light at the end of the tunnel for unsupervised approaches too.

Let's begin with my own requirements, which are mainly created to only keep the fun problem-solving left.

- The end-product must be unsupervised
  - No manually annotated data
  - No heuristic applied (at least in first iteration)
- It should be light enough to run on a Raspberry Pi later on (hopefully on the JVM to keep it simple with my back-end)
- Must be Swedish all the way through - no translations (English models you can transfer knowledge from tends to be stronger, but I want to keep this fun!)

With this in mind I set out to build my own FAQ search engine.

**What is required to answer questions using a FAQ?** We need to find the most relevant Q/A to the question posed. 

**How do we do this?** There is numerous types of ways to do this unsupervised. I'll account for a few here:

1. Latent Dirichlet Allocation (LDA) which is a way to find topics through clever statistical analysis (basically soft clusters of documents)
2. Embedding and [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity), find the distance between the two arrays of numbers in the embedded space. One can also apply Euclidean Distance which isn't especially good because of [Curse of Dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality). Other possible approaches includes [Word Mover Distance](https://arxiv.org/pdf/1912.00509.pdf). 
3. Simple word counting and Bag of Words

### Tools Chosen
After a little research I found a few tools which fit my need. 

**fastText**

fastText that came out of Facebook AI Research (FAIR) and [this](https://arxiv.org/abs/1607.04606) paper. It's a type of Word Embeddings where also subwords are embedded through ngrams of characters, this means that we are able to embedd words that are out of vocabulary, which can be the reason because of either misspelling or just a missing word.
On their [homepage](https://fasttext.cc/) they have a plethora of models including a Swedish one that has been derived from Wikipedia, pretty awesome!

**Smooth Inverse Frequency**

Smooth Inverse Frequency (SIF) is an algorithm to embed sentences which was proposed in ["A Simple but Tough-To-Beat Baseline for Sentence Embeddings"](https://openreview.net/pdf?id=SyK00v5xx) in 2017. In its essence they propose to embed the sentence using a weighted average and thereafter modify them a bit using PCA/SVD.

**Folkhälsomyndigheten FAQ**

Finally I need the FAQ to use, in my case it's Covid-19 FAQ from Folkhälsomyndigheten. It was parsed into pandas dataframes using requests & BeautifulSoup4 (bs4).

### Final Result
So after all this was figured out I sat down an afternoon and cooked some code together, the result ended up more impressive than I had imagined. The questions posed are being responded with pretty good results. I'm especially impressed by question about _astma_, _son_ and _regler_. Here's a few of them:



```
> Hur sjuk blir jag?

Hur sjuk blir man av covid-19? - 0.98
Hur länge är man sjuk av covid-19? - 0.97
Hur lång är inkubationstiden? - 0.81
```

```
> Hur vet jag om det är astma?

Hur vet jag om mina symtom beror på pollenallergi eller på covid-19? - 0.63
Hur sjuk blir man av covid-19? - 0.53
Hur länge är man sjuk av covid-19? - 0.53
```

```
> Hur förklarar jag corona för min son?

Hur pratar man med barn om det nya coronaviruset? - 0.58
Hur lång är inkubationstiden? - 0.53
Hur sjuk blir man av covid-19? - 0.49
```

```
> Hur minskar vi spridningen i sverige?

Hur gör ni för att mäta förekomsten av covid-19 i samhället? - 0.65
Hur övervakar ni på Folkhälsomyndigheten spridningen av covid-19? - 0.57
Hur stor är dödligheten till följd av covid-19? - 0.56
```

```
> Vad för regler finns?

Vad gäller för olika verksamheter? - 0.76
Vad gäller för handeln? - 0.75
Vad är covid-19? - 0.71
```

One can directly note the correlation of the beginning. It seems like the first word has a high correlation with the most similar question. Weird. Removing stop words could probably improve this, but that'd be for the second implementation.

### Further improvements for iteration 2, 3 and beyond!
**Pre-processing**

As mentioned right above we can apply some basic pre-processing  such as removing stop words. In reality this should be handled by SIF but looking at our similarity scores there's a 1-1 relation between the first word of the sentence. 

Other improvements worth trying out is lemmatizing or stemming the words ("cutting them to the root" in simple terms) and further using a better tokenization is worth trying out (currently splitting on whitespace). _spaCy_ offers a strong tokenizer, but I haven't tried it out for Swedish yet. Once again _fastText_ should handle this but it's worth trying out if it improves or keep the result at the same level.

**Different Embedding Techniques**

There exist a certain Sentence Embedding that's basically made for this task - MULE (Multimodal Universal Language Embeddings). MULE is even multilingual but unfortunately they're not able to embed Swedish so we'd require a translation from Swedish to one of the 16 languages supported by MULE. This means that it is out of the question because of my requirements, but could still be fun to check out. 

Other embeddings such as FLAIR (by Zalando), BERT (using BERT-as-a-service) or even training my own embeddings (perhaps using StarSpace) could prove interesting also.

**Completely other technique**

I mentioned first of all LDA, and I think LDA could be interesting. Most often LDA is applied to larger documents but as with everything it is never wrong to try out and verify the results. 

Supervised approaches would certainly be able to show us some good performance but that requires annotating data in one way or another which is a boring task - but very important. Perhaps I'll revisit and label some data, with todays Transfer Learning we can achieve higher accuracy with less data using other pre-trained  Language Models such as BERT or Multifit (from Ulmfit).

### Ending words
This was a really fun task and I'm happy that I tried it out. I'm sure I'll revisit and improve it further by applying some of the possible improvements. Further I think I might actually try to do this for all FAQs available by our authorities to create a "Multi FAQ" which could prove pretty cool. With more data the results should also be better.

And as an ending note my model ended up using 2.5-3 GB of memory during run-time which means it's possible to run on my Raspberry Pi 4! Further reduction of size can be done by removing the most uncommon words in the vocabulary (vocab is 2M words, which is very large). I applied a dimension reduction using the built in version of _fastText_ (ending up using d=100 and still achieving good search results).

The implementation is available right beneath. Or at [Londogard GitHub](https://github.com/londogard/).

Thanks for this time, I'll be back with more! 

-----

### Code
First installation of libraries required.

In [1]:
%%capture
!pip install gensim --upgrade
!pip install -U fse
!pip install stanza

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.spatial import distance # quick vector comparison with cdist + cosine
from tqdm import tqdm
tqdm.pandas()

from pathlib import Path

import os
import random 
import operator 
import regex as re

# gensim + fasttext
from gensim.models.fasttext import FastText, load_facebook_vectors
from gensim.models import KeyedVectors

import stanza

from bs4 import BeautifulSoup
import requests

In [ ]:
# !wget # https://drive.google.com/open?id=1vaWtiSlRAZ3XCdtnSce_6dwQ0T5x0OEJ
stanza.download('sv')
# This is the model with reduced dimensions. Couldn't upload it elsewhere, simplest is to move it to your drive and use the drive tool

In [10]:
wv = load_facebook_vectors('~/git/nlp-projects/models/cc.sv.100.bin')

## Getting the FAQ from Folkhälsomyndigheten


In [8]:
def get_title(p):
  return p.find(class_='accordion__item__title__text').span.text

# OBS: For now removing hrefs!
def get_answer(p):
  lines = p.find('div', class_='textbody').find_all('p')
  lines = ' '.join([l.text for l in lines[:-1] if l.a == None])
  return lines


r = requests.get("https://www.folkhalsomyndigheten.se/smittskydd-beredskap/utbrott/aktuella-utbrott/covid-19/fragor-och-svar/")
html = r.text

soup = BeautifulSoup(html, "html.parser")

divs = soup.find_all('li', class_='accordion__item toggle')
with open ('covid.tsv', 'w') as f:
  f.write('question\tanswer\n')
  for d in divs:
    f.write(f"{get_title(d)}\t{get_answer(d)}\n")

In [9]:
df = pd.read_csv('covid.tsv', sep='\t')
df.head()

,question,answer
0,Vad innebär de nationella föreskrifterna och a...,De skärpta nationella föreskrifterna och allmä...
1,Vilka regler gäller för resor under jul- och n...,Du bör avstå från att resa så mycket som möjli...
2,Vilka rekommendationer gäller för jul- och nyå...,Vinterns högtider bör firas i en mindre krets....
3,Ska alla som precis kommit eller kommer från S...,Ja. Brittiska myndigheter har upptäckt en ny v...
4,Vilka olika tester finns det för covid-19?,Det finns test som kan visa om du har en pågåe...


In [6]:
df['sent_question'] = df['question'].apply(lambda x: x.split(' '))
df.head()

,question,answer,sent_question
0,Vad är coronavirus?,Det finns ett stort antal virus som tillhör co...,"[Vad, är, coronavirus?]"
1,Vad är covid-19?,Covid-19 är det officiella namnet för sjukdome...,"[Vad, är, covid-19?]"
2,Vad skiljer covid-19 från säsongsinfluensa?,"Det är svårt att jämföra en helt ny sjukdom, e...","[Vad, skiljer, covid-19, från, säsongsinfluensa?]"
3,Vilka är symtomen på covid-19?,Sjukdomen är hos de flesta en luftvägsinfektio...,"[Vilka, är, symtomen, på, covid-19?]"
4,Hur vet jag om mina symtom beror på pollenalle...,Både covid-19 och pollenallergi kan ge milda f...,"[Hur, vet, jag, om, mina, symtom, beror, på, p..."


In [4]:
df['embedded_q'] = df['sent_question'].apply(lambda x: [wv.get_vector(w) for w in x])
df['embedded_avg'] = df['embedded_q'].apply(lambda x: np.mean(x, axis=0))
df.head()

NameError: name 'wv' is not defined

In [0]:
question = "Hur påverkas jag med astma?"
q_embedded = np.mean([wv.get_vector(w) for w in question.split()], axis=0)

dists = distance.cdist(df['embedded_avg'].values.tolist(), [q_embedded], 'cosine')
dists = np.array([d[0] for d in dists])
clostest_questions = np.argsort(-dists)[:5]
df.iloc[clostest_questions.tolist()]

Pretty weak to be honest...

## FSE: Fast Sentence Embeddings using Cython, Optimizations & SIF (Smooth Inverse Frequency)

Enter FSE, a library that makes it incredibly easy to handle large amount of text quickly (overkill for this application). It also applies SIF which is an interesting approach, read about it in their paper: https://openreview.net/pdf?id=SyK00v5xx

In [13]:
nlp = stanza.Pipeline(lang='sv', processors='tokenize')
df['question_tokens'] = df['question'].apply(lambda x: [token.text for sentence in nlp(x.lower()).sentences for token in sentence.tokens])

2020-05-26 18:50:55 INFO: Loading these models for language: sv (Swedish):
| Processor | Package   |
-------------------------
| tokenize  | talbanken |

2020-05-26 18:50:55 INFO: Use device: cpu
2020-05-26 18:50:55 INFO: Loading: tokenize
2020-05-26 18:50:55 INFO: Done loading processors!


In [14]:
df['question_tokens']

0                             [vad, är, coronavirus, ?]
1                                [vad, är, covid-19, ?]
2     [vad, skiljer, covid-19, från, säsongsinfluens...
3                [vilka, är, symtomen, på, covid-19, ?]
4     [hur, vet, jag, om, mina, symtom, beror, på, p...
                            ...                        
76    [varför, stänger, vi, inte, för-, och, grundsk...
77    [hur, pratar, man, med, barn, om, det, nya, co...
78                            [vad, är, en, pandemi, ?]
79    [det, nya, coronaviruset, sprids, i, första, h...
80    [måste, jag, följa, ett, allmänt, råd, eller, ...
Name: question_tokens, Length: 81, dtype: object

In [15]:
from fse import IndexedList
s = IndexedList(df['question_tokens'].tolist())

In [16]:
from fse.models.average import FAST_VERSION, MAX_WORDS_IN_BATCH
print(MAX_WORDS_IN_BATCH)
print(FAST_VERSION)

from fse.models import uSIF
model = uSIF(wv, workers=4, lang_freq="sv")
model.train(s)

10000
1


(81, 772)

In [24]:
q = "kan min hund bli sjuk"
text = [token.text for sentence in nlp(q.lower()).sentences for token in sentence.tokens]
model.sv.similar_by_sentence(text, model=model, indexable=s.items)

[(['kan',
   'covid-19',
   'smitta',
   'mellan',
   'djur',
   'och',
   'människa',
   'och',
   'kan',
   'mitt',
   'husdjur',
   'smittas',
   'av',
   'viruset',
   '?'],
  33,
  0.5361529588699341),
 (['hur',
   'ska',
   'vi',
   'bete',
   'oss',
   'när',
   'någon',
   'i',
   'hushållet',
   'är',
   'sjuk',
   'i',
   'misstänkt',
   'eller',
   'konstaterad',
   'covid-19',
   '?'],
  48,
  0.52496737241745),
 (['kan',
   'man',
   'bli',
   'smittad',
   'av',
   'en',
   'person',
   'som',
   'har',
   'covid-19',
   'men',
   'som',
   'inte',
   'har',
   'några',
   'symtom',
   '?'],
  28,
  0.4506794214248657),
 (['kan',
   'viruset',
   'smitta',
   'till',
   'människa',
   'via',
   'post',
   'och',
   'paket',
   '?'],
  31,
  0.4159470498561859),
 (['bör', 'jag', 'som', 'symtomfri', 'jobba', 'hemma', '?'],
  51,
  0.39490681886672974),
 (['jag',
   'har',
   'förlorat',
   'en',
   'närstående',
   'och',
   'känner',
   'djup',
   'sorg.',
   'vad',
   'ka

# TODOs

- [ ] Prettify
- [ ] Show improvement through
- [ ] Average -> FSE
- [ ] Split -> Stanza tokenization
- [ ] Cased -> Lowercase
- [ ] Lemmatizing ???
- [ ] Stop words ???
- [ ] Supervised required to get further!
